In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql import DataFrame
import pyspark.sql.functions as F

In [2]:
# get or create Spark session

app_name = "spark-review"
spark = SparkSession.builder.appName(app_name).getOrCreate()

In [3]:
rs_file ="/FileStore/tables/ratings_and_sentiments.csv"
df_rs = spark.read \
    .format("com.databricks.spark.csv") \
    .options(header='true', inferSchema="true") \
    .load(rs_file)
df_rs.printSchema()

root
-- coffee_shop_name: string (nullable = true)
-- review_text: string (nullable = true)
-- rating: string (nullable = true)
-- num_rating: integer (nullable = true)
-- cat_rating: string (nullable = true)
-- bool_HIGH: integer (nullable = true)
-- overall_sent: integer (nullable = true)
-- vibe_sent: string (nullable = true)
-- tea_sent: integer (nullable = true)
-- service_sent: integer (nullable = true)
-- seating_sent: integer (nullable = true)
-- price_sent: integer (nullable = true)
-- parking_sent: string (nullable = true)
-- location_sent: integer (nullable = true)
-- alcohol_sent: integer (nullable = true)
-- coffee_sent: string (nullable = true)
-- food_sent: string (nullable = true)
-- hours_sent: integer (nullable = true)
-- internet_sent: integer (nullable = true)
-- local_sent: integer (nullable = true)
-- _c20: string (nullable = true)
-- _c21: string (nullable = true)
-- _c22: string (nullable = true)
-- _c23: string (nullable = true)
-- _c24: string (nullable = true)
-- _c25: string (nullable = true)
-- _c26: string (nullable = true)
-- _c27: string (nullable = true)
-- _c28: string (nullable = true)
-- _c29: string (nullable = true)
-- _c30: string (nullable = true)
-- _c31: string (nullable = true)
-- _c32: string (nullable = true)
-- _c33: string (nullable = true)
-- _c34: string (nullable = true)
-- _c35: string (nullable = true)
-- _c36: string (nullable = true)
-- _c37: string (nullable = true)

In [4]:
import boto3

secret_name = "ut/postgres/db"
region_name = "us-east-2"
access_key = "YOUR ACCESS HERE"
secret_key = "YOUR KEY HERE"

session = boto3.session.Session(aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=region_name)
client = session.client('secretsmanager')
secret_value = client.get_secret_value(SecretId=secret_name)
# secret_value

In [5]:
import json
def get_connection(secret_value):
  return json.loads(secret_value['SecretString'])
# get_connection(secret_value)

In [6]:
connection = get_connection(secret_value)

# Postgres credentials
jdbcHostname = connection['host']
jdbcPort = connection['port']
jdbcDatabase = "postgres"
dialect = "postgresql"
jdbcUsername = connection['username']
jdbcPassword = connection['password']

jdbcUrl = f"jdbc:{dialect}://{jdbcHostname}:{jdbcPort}/{jdbcDatabase}"
connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "org.postgresql.Driver" 
}
# for mysql driver = com.mysql.jdbc.Driver

In [7]:
# CREATE TABLE review_ratings (
#   label TEXT,  -- This will be the review text
#   ratings INTEGER,
#   review_length INTEGER -- length of the review text
# )

In [8]:
table = "coffee_rating"
mode = "overwrite"

df_rs = df_rs.withColumn("label", F.regexp_extract("review_text", r'\d+/\d+/\d+(?:\s)(.*)', 1))
df_rs.head()

Out[14]: Row(coffee_shop_name='The Factory - Cafe With a Soul ', review_text='11/25/2016 1 check-in Love love loved the vibe! Every corner of the coffee shop had its own style and there were swings!!! I ordered the matcha coffee and it was muy fantastico! Ordering and getting my drink were pretty streamlined. I ordered on an iPad which included all beverage selections that ranged from coffee to alcohol desired level of sweetness and a checkout system. I got my coffee within minutes! I was hoping for a typical heart or feather on my coffee but found myself listing out all the possibilities of what the vibe may be. Any ideas?', rating=' 5.0 star rating ', num_rating=5, cat_rating='HIGH', bool_HIGH=1, overall_sent=4, vibe_sent='3', tea_sent=0, service_sent=0, seating_sent=0, price_sent=0, parking_sent='0', location_sent=0, alcohol_sent=1, coffee_sent='3', food_sent='0', hours_sent=0, internet_sent=0, local_sent=0, _c20=None, _c21=None, _c22=None, _c23=None, _c24=None, _c25=None, _c26=None, _c27=None, _c28=None, _c29=None, _c30=None, _c31=None, _c32=None, _c33=None, _c34=None, _c35=None, _c36=None, _c37=None, label='1 check-in Love love loved the vibe! Every corner of the coffee shop had its own style and there were swings!!! I ordered the matcha coffee and it was muy fantastico! Ordering and getting my drink were pretty streamlined. I ordered on an iPad which included all beverage selections that ranged from coffee to alcohol desired level of sweetness and a checkout system. I got my coffee within minutes! I was hoping for a typical heart or feather on my coffee but found myself listing out all the possibilities of what the vibe may be. Any ideas?')

In [9]:
table = "review_ratings"
mode = "overwrite"

df_rs.groupBy("label") \
  .agg(F.mean("num_rating").alias("ratings")) \
  .withColumn("review_length", F.length("label")) \
  .write \
  .jdbc(jdbcUrl, table, mode, connectionProperties)